

# `neos`: End-to-End-Optimised Summary Statistics for High Energy Physics

## Introduction

**Summary statistics** are typically low-dimensional quantities
calculated from high-dimensional data, often for the purpose of making
any following inference stage easier to compute. They find widespread
use in high-energy physics (HEP) data analysis for this reason, since
HEP data, e.g. that produced at the Large Hadron Collider, is
characterised by both its high dimensionality ($\mathcal{O}(10^8)$
detector read-out channels) as well as the intractability of computing
its probability $p(x|\theta)$ under a given physics hypothesis $\theta$.
A notable example of such a quantity would be the invariant mass of the
Higgs boson, which was used in its discovery in 2012 [@2012].

Thanks to the presence of high-quality physics simulators, effective
machine-learning based summary statistics can be formed using simulated
data, typically being trained using classification-driven objectives to
maximally distinguish new physics from background processes. While these
objectives can be shown to produce optimal results in simplified
hypothesis testing settings, the presence of systematic uncertainties in
the data may cause classification objectives to become misaligned with
physics objectives, e.g. the expected sensitivity to a signal. This
means that algorithms trained in this way may under-perform when
integrated into the actual physics analysis pipeline, motivating the
search for objectives that can better adapt to data with high systematic
uncertainty.

To combat this, we propose to optimise the summary statistic directly on
physics objectives by formulating an *end-to-end-differentiable analysis
pipeline*, including density estimation of the summary statistic with
histograms, and statistical inference using standard HEP prescriptions.
The training can then based on physics-oriented metrics, such as the
expected sensitivity, which is based on the profile likelihood ratio: a
quantity capable of capturing systematic effects.

## Making HEP Analysis Differentiable

Given a pre-filtered dataset, an analysis pipeline in HEP involves the
following stages:

1.  Construction of a learnable 1-D summary statistic from data (with
    parameters $\phi$)

2.  Binning of the summary statistic, e.g. through a histogram

3.  Statistical model building, using the summary statistic as a
    template

4.  Calculation of a test statistic, used to perform a frequentist
    hypothesis test of signal versus background

5.  A $p$-value (or $\mathrm{CL_s}$[^1] value) resulting from that
    hypothesis test, used to characterise the sensitivity of the
    analysis

We can express this workflow as a direct function of the input dataset
$\mathcal{D}$ and observable parameters $\phi$:

$$\label{eq:decomp}
    \mathrm{CL_s} = f(\mathcal{D},\phi) = (f_{\mathrm{sensitivity}} \circ f_{\mathrm{test\,stat}} \circ f_{\mathrm{likelihood}}  \circ f_{\mathrm{histogram}}  \circ f_{\mathrm{observable}})(\mathcal{D},\phi).$$

In the common case where $f_{\mathrm{observable}}$ is a neural network,
it seems possible to optimise this composition end-to-end, i.e. *train
the network to directly optimise the analysis sensitivity*. This is
exactly the task that `neos` sets out to accomplish, with a full
workflow detailed in [\[fig:pipe\]](#fig:pipe){reference-type="autoref"
reference="fig:pipe"}. To train this network by gradient descent, our
choice of $\mathrm{loss} = \mathrm{CL_s}$ requires us to be able to
calculate $\partial \,\mathrm{CL_s} / \partial \phi$. However, this is a
stronger condition than it seems, and in fact necessitates the
differentiablility of *each individual analysis step* via the chain rule
applied to [\[eq:decomp\]](#eq:decomp){reference-type="autoref"
reference="eq:decomp"}.



Owing to the fact that neural networks are already differentiable, the
last term $\partial f_{\mathrm{observable}}/{{\partial \phi}}$ isn't an
issue, but none of the rest of the steps are differentiable by default.
The following sections detail solutions for calculating the gradient of
each intermediate step.

### Binned density estimation (histograms)

Histograms are discontinuous by nature. They are defined for 1-D data as
a set of two quantities: intervals (or *bins*) over the domain of that
data, and counts of the number of data points that fall into each bin.
For small changes in the underlying data distribution, bin counts will
either remain static, or jump in integer intervals as data migrate
between bins, both of which result in ill-defined gradients. We address
this inherent non-differentiability through implementing a
differentiable surrogate: a histogram based on a *kernel density
estimate* (KDE).

A KDE is a "non-parametric\" density estimate based on defining a kernel
function $K$ centred on each data point. Then, the density at an
evaluation point $x$ is the average of the contributions of each kernel
function at that point.

a with the full density given as $p(t) = 1/n\sum_i K(t,t_i)$. Normally,
a popular kernel function choice is the standard normal distribution,
which comes with a parameter called the **bandwidth** that affects the
smoothness of the resulting density estimate.

Coming back to gradients: in our case, the data $t_i$ we construct the
density estimate over are themselves functions of the summary statistic
parameters, i.e. $t_i = f(x_i;\phi)$. The resulting density estimate
$p(t|\phi)$ will then be differentiable as long as the kernel $K$ is
differentiable with respect to $t_i$, and by extension with respect to
$\phi$. To extend this differentiability in a binned fashion, we can
accumulate the probability mass of the KDE within the bin edges of the
original histogram -- equivalent to evaluations of the Gaussian
cumulative density function -- to convert $p(t|\phi)$ to a **binned KDE
(bKDE)**, i.e. a set of discrete per-bin probabilities $p_i(\phi)$.

In the limit of vanishing bandwidth, the bKDE recovers the standard
histogram, but gradients become susceptible to high variance. Increasing
the bandwidth can alleviate this, but at the cost of introducing a bias.
There is then a trade-off between decreasing the bandwidth enough to
minimise this bias, and increasing it enough to guarantee gradient
stability[^2]. We can see a demonstration of this behaviour in
[\[fig:hist\]](#fig:hist){reference-type="autoref"
reference="fig:hist"}, where the bandwidth is tuned relative to the bin
width.

![Illustration of the bias/smoothness tradeoff when tuning the bandwidth
of a bKDE, defined over 200 samples from a bi-modal Gaussian mixture.
All distributions are normalised to unit area. The individual kernels
that make up the KDE are scaled down for
visibility.](relaxed_hist.pdf){#fig:hist width="\\textwidth"}

### Likelihood modelling

A popular framework to build statistical models for binned observations
based on "template\" histogram data is HistFactory [@hifa], used widely
used across HEP. Thankfully, the resulting log-likelihood function
$p_\mathbf{h}(x)$ is differentiable with respect to both the observed
data $x$ and histogram data $\mathbf{h}$ from which the model is
constructed. However, these gradients only recently became readily
accessible in software, owing to the development of `pyhf`
[@pyhf; @pyhf2]: a Python package for building these likelihoods that
leverages automatic differentiation.

### Test statistics (profile likelihood ratio)

A number of test statistics are typically used in HEP[^3], all of which
build on the profile likelihood ratio. For likelihood function $p$,
input data $x$, parameters of interest (POIs) $\mu$, and nuisance
parameters $\psi$, we define the profile likelihood ratio as

$$\begin{aligned}
    \lambda_\mu(x) = \frac{p(x|\mu,%
    \settoheight{\dhatheight}{\ensuremath{\hat{\psi}}}%
    \addtolength{\dhatheight}{-0.35ex}%
    \hat{\vphantom{\rule{1pt}{\dhatheight}}%
    \smash{\hat{\psi}}}(\mu))}{p(x|\hat{\mu},\hat{\psi})},\end{aligned}$$
where $\hat{\mu},\hat{\psi}$ denote the maximum-likelihood parameters,
whilst $%
    \settoheight{\dhatheight}{\ensuremath{\hat{\psi}}}%
    \addtolength{\dhatheight}{-0.35ex}%
    \hat{\vphantom{\rule{1pt}{\dhatheight}}%
    \smash{\hat{\psi}}}(\mu)$ denotes the maximum-likelihood estimate of
$\psi$ at fixed values of $\mu$. A test statistic $q_\mu$ is then built
upon the profile likelihood, following prescriptions detailed in
[@asymptotics]. Our physics objective, i.e. the expected
$\mathrm{CL}_s$, is computed from distributions of $q_\mu$ for the null
and alternative hypotheses, which correspond to choosing different
values of $\mu$. In asymptotic theory, the null is distributed as a
$\chi^2$-distribution, while the alternative follows a non-central
$\chi^2_\mathrm{nc}(x,\Lambda^2)$ distribution, with $\Lambda^2$ given
by $$\Lambda^2 = (\mu-\mu')^2/\sigma_{\hat{\mu}}^2,$$ where $\mu$,
$\mu'$ are the POI values corresponding to the null and alternative
hypothesis respectively, and $\sigma_{\hat{\mu}}$ is the variance for
the maximum likelihood estimate of the POIs. Our goal of minimising the
expected $\mathrm{CL}_s$ is then equivalent to both maximising
$\Lambda^2$ and minimising $\sigma_{\hat{\mu}}$.

While the INFERNO approach uses the Fisher Information to estimate and
minimise $\sigma_{\hat{\mu}}$, we take the route of computing
$\Lambda^2$ as the test statistic value for the "Asimov dataset" of the
alternative hypothesis. As described in [@asymptotics], the use of
Asimov test statistics is known to yield better approximations of the
true sampling distributions, and therefore of the final objective
$\mathrm{CL_s}$. However, the drawback of this approach is the inclusion
of optimisation routines such as the computation of $%
    \settoheight{\dhatheight}{\ensuremath{\hat{\psi}}}%
    \addtolength{\dhatheight}{-0.35ex}%
    \hat{\vphantom{\rule{1pt}{\dhatheight}}%
    \smash{\hat{\psi}}}$, which are not a-priori differentiable in an
efficient manner. We thus explore methods for derivatives of
optimisation routines that only require gradient information when
optimisation has converged.

#### Implicit differentiation of optimisation routines:

Optimisation algorithms find best-fit values $\hat{\theta}$ for a given
objective function $f$. In the case of the profile likelihood, the
objective $f$ is the likelihood function, which implicitly depends on
the summary statistic parameters $\phi$; we denote this by $f = f_\phi$,
and extend this dependence to the best-fit parameters
$\hat{\theta}(\phi)$. Iterative optimisation finds $\hat{\theta}$ by
starting at a initial value $\theta_0$ and repeatedly updating it
through a iteration step $\hat{\theta}_{i+1}=u(f_\phi, \theta_i)$ until
a terminating condition is satisfied. An explicit expression for
$\hat{\theta}(\phi)$ capturing the dependence on $\phi$ is not readily
available. Despite this, we can still compute gradients of
$\hat{\theta}$ by noting that at the solution $\theta_i = \hat{\theta}$,
the following relation holds for any $\phi$:
$$\hat{\theta} = u(f_\phi,\hat{\theta}) ~\Rightarrow~ g(\phi,\hat{\theta}) = u(f_\phi,\hat{\theta}) - \hat{\theta} = 0,$$
From here, we can leverage a powerful result from the *implicit function
theorem* that locally guarantees the existence of a continuously
differentiable function $\hat{\theta}(\phi)$, with gradients given by
$$\label{eq:fi}
    \frac{\partial\hat{\theta}(\phi)}{\partial\phi} = -\left[\frac{\partial g}{\partial \theta}\right]^{-1}  \frac{\partial g}{\partial \phi} = \left[I - \frac{\partial u}{\partial \theta}\right]^{-1} \frac{\partial u}{\partial \phi}$$
This is our saving grace: by evaluating derivatives of *just the update
step* with respect to the optimised parameters $\theta$ and the
parameters implicitly defining the objective $\phi$, we can derive
gradients of the resulting optimisation solution without unrolling all
iterations. Implicit differentiation through optimisation procedures
like this has been implemented in software by the authors of the
`jaxopt` package [@jaxopt], which we use in our experiments. We refer
the reader to Ref. [@implicit] for more detail on the connection between
[\[eq:fi\]](#eq:fi){reference-type="autoref" reference="eq:fi"} and
automatic differentiation.

This result lets us compute the non-centrality parameter $\Lambda^2$ of
the hypothesis test (and as a corollary, the expected $\mathrm{CL_s}$)
in a fully differentiable manner, providing the final building block of
an end-to-end differentiable analysis pipeline as described in the
beginning of this section.

## Demonstrating `neos`: End-to-end Analysis Optimisation

We demonstrate the principle of end-to-end optimisation by training a
neural network optimised directly on the physics sensitivity, expressed
as the expected $\mathrm{CL_s}$ value of a `pyhf` model.

### Gaussian blob experiment

![Analysis metrics evaluated on a test set when running the `neos`
pipeline with a variety of loss functions, averaged over 7
initialisations of the neural network weights. **Left**: the expected
$\mathrm{CL_s}$ with a null hypothesis of no signal. **Middle**: the
uncertainty on the signal strength $\mu$. **Right**: deviation of the
nuisance parameter uncertainty $\gamma$ from its nominal value of
1.](metric_comparison_fewer.pdf){#fig:metrics width="\\textwidth"}

We consider an example where we generate toy physics data $(x, y)$ from
2-D Gaussian blobs with covariances equal to the 2-D identity matrix
$I_2$.

The dataset $\mathcal{D}$ consists of signal
${s} \sim \mathcal{N}(m_s, I_2)$, background
$b \sim \mathcal{N}(m_b, I_2)$, and two variations of the background
data $b\mathrm{_{up}}, {b_{\mathrm{down}}}\}$ that mimic simulating the
background process after varying an imagined physical parameter $\Psi$
up and down by one standard deviation, which is a common situation
encountered in HEP. In practice, $\Psi$ is taken to influence the mean
of the background distribution implicitly through generating blob data
with different means, i.e.
${b\mathrm{_{up}}} \sim \mathcal{N}(m_{b{\mathrm{up}}}, I_2)$,
${b_{\mathrm{down}}} \sim \mathcal{N}(m_{b{\mathrm{down}}}, I_2)$.

After dataset generation, as well as splitting into train and test
datasets, the workflow then follows the steps outlined in
[\[fig:pipe\]](#fig:pipe){reference-type="autoref"
reference="fig:pipe"}. Results from running this experiment can be found
in [\[fig:metrics\]](#fig:metrics){reference-type="autoref"
reference="fig:metrics"}, which compares `neos` to three baseline loss
functions:

-   Binary cross-entropy (BCE): signal vs. nominal background

-   BCE, but using up/down systematic variations as data augmentation
    for the background sample. This is a powerful baseline that
    maximally isolates the signal points from all background variations.

-   INFERNO: using the signal strength uncertainty $\sigma_\mu^2$ as the
    objective, calculated from the Hessian of the likelihood via
    automatic differentiation.

Three metrics are shown: the expected $\mathrm{CL_s}$, the parameter of
interest ($\mu$) uncertainty, and the nuisance parameter ($\gamma$)
uncertainty, all evaluated on the test set[^4]. All metrics are averaged
over seven different initialisations of the weights $\phi$.
Hyperparameters are given in [Appendix A](#app1).

We can see that `neos` is able to reach sensitivities lower than any
other loss function (left plot). Moreover, it is able to gradually
improve the uncertainty on the parameter of interest (middle plot)
without any additional tuning or regularisation, far outperforming
binary cross-entropy in this regard. When mixing in the up/down
systematic variations as part of the background class when using BCE, we
are able to get similar performance to `neos` with lower $\mu$
uncertainty around epoch 4, but `neos` is able to improve to match this
with more training, all while retaining a lower $\mathrm{CL_s}$. INFERNO
is able to achieve the best uncertainty on $\mu$, but fails to reach the
same expected sensitivity as other methods.

We additionally examine the uncertainty on $\gamma$ (right plot), also
taken from the Fisher information matrix. It indicates whether the
analysis is able to determine the systematic effect on the background
shape in a way that differs from the constraint on $\gamma$ provided by
the up/down variations. This is typically undesirable behaviour, as it
indicates a strong dependence of the summary statistic on the nuisance
parameters. Here, we see that `neos` and BCE (with aug.) are able to
keep this metric fairly low, while BCE and INFERNO show signs of over-
or under- constraining $\gamma$.

## Discussion and Future Directions

One of the more immediate concerns for using `neos` in practice is the
issue of scaling. Training times compared to BCE in the toy problem are
a factor of 3 or so higher, since the processing of one batch
corresponds to running the all the downstream parts of a physics
analysis. In addition, the batch size needs to be large enough to
faithfully represent the analysis, hopefully avoiding situations with
empty bins. This is an issue that is currently being investigated; a
medium/long term goal would be to scale `neos` in application to open
data from the major LHC experiments, such as ATLAS and CMS, allowing the
probing of how the performance benefits highlighted in section 3 scale
to realistic problems with many sources of uncertainty.

Despite the focus on summary statistics, the work done in enabling the
differentiability of HEP analysis opens up a variety of new approaches
in optimising any free parameters end-to-end. For example, one could
imagine optimising the pre-selection stage that occurs when filtering
data prior to inference, since one can differentiably approximate a cut
with a sigmoid function. Moreover, one could optimise the binning of an
observable for the best sensitivity, given a fixed number of bins.

An additional, exploratory direction is considering the use of a
multi-term objective function with weighted components. The software
released alongside this work already enables objectives of the form
$a_0{\mathrm{CL_s}} + a_1{\sigma_\mu} + a_2{(1-\sigma_\gamma)^2} + \dots$,
which could offer regularisation effects to steer away from pathologies
that happen to satisfy one metric only. Work in this direction is also
ongoing.

A future comparison to black box methods such as Bayesian optimisation
would also be desirable, as they do not require the use of
approximations to enable optimisation, but still can target the physics
goals of choice.

## Conclusion

The typical HEP analysis workflow has been made differentiable, and a
use case for it in the systematic-aware optimisation of a neural network
has been demonstrated. Comparison was made to the more standard approach
of using binary cross-entropy, as well as more competitive baselines
like systematic data augmentation and INFERNO. Improvements were shown
in both the expected analysis sensitivity and the properties of the
likelihood function. Accompanying software to run the experiments
detailed in Section 3 can be found at [@neos], as well as a toolbox
called `relaxed` [@relaxed] that enables the isolated or combined use of
the differentiable components used to assemble `neos`.

We would like to thank the authors of INFERNO (Pablo de Castro, Tommaso
Dorigo) for initial productive discussions. We also extend thanks to
Kyle Cranmer, Alexander Held, Giordon Stark, and Matthew Feickert for
conceptual and technical support in the development of this project.
Visualisations are produced using the `matplotlib` package [@mpl]. LH is
supported by the Excellence Cluster ORIGINS, which is funded by the
Deutsche Forschungsgemeinschaft (DFG, German Research Foundation) under
Germany's Excellence Strategy - EXC-2094-390783311. NS completed this
work with financial support from Insights, which is funded by the
European Union's Horizon 2020 research and innovation programme, call
H2020-MSCA-ITN-2017, under Grant Agreement n. 765710. NS extends
gratitude to friends and colleagues that indirectly facilitated this
work, particularly to those in the "noonch" lunch group at CERN.

##  {#app1}

Hyperparameters for the Gaussian blob study in Section 3:

-   10000 data points, split evenly between all four blobs,

-   3-layer neural network of size (1024, 1024, 1),

-   Training with Adam optimiser, learning rate 1e-3,

-   Adam optimiser also used in maximum likelihood fits with learning
    rate 1e-2,

-   $m_\mathrm{s}=(-1, 1)$, $m_\mathrm{b}=(2.5, 2)$,
    $m_\mathrm{bup}=(-2.5, -1.5)$, $m_\mathrm{bdown}=(1, -1)$,

-   Multiplicative histogram scale factors: signal scale=2, background
    scale=10, global scale=10,

-   ReLU activations, with sigmoid activation on the final layer,

-   15 epochs, with a batch size of 2000.

# References {#references .unnumbered}

[^1]: $\mathrm{CL_s}$ is a modification of the p-value that protects
    against rejecting the null hypothesis when the test is not sensitive
    to the alternative hypothesis (e.g. through largely overlapping test
    statistic distributions).

[^2]: How low is low enough when tuning? This relationship between bias
    and gradient stability is heavily impacted by the number of data
    points, and also by the width of the intervals, with more
    exploration of this planned.

[^3]: A full list of the different test statistics and their purposes
    can be found in [@asymptotics].

[^4]: During test evaluation and training with binary cross-entropy, no
    differentiable approximations are used.
